In [8]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
# install sentence transformers if you don't have
#import libraries
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
#load model
model  = SentenceTransformer("bert-base-uncased")
# see the details of model with print 
print(model)

No sentence-transformers model found with name /Users/remicarvalot/.cache/torch/sentence_transformers/bert-base-uncased. Creating a new one with MEAN pooling.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


We will report each results in a same dataframe to plot the different results in function of parameters.

In [62]:
parameters = ['model_name','model_type','method_preprocessing','embedding_model','chunk_size','chunk_overlap','context_length','chunk_technique','temperature','gpu_layers','search_kwargs','size_model','size_answer','reorder','quantization','device','rate','rate_table','rate_text','time']

In [63]:
dataframe_report = pd.DataFrame(columns=parameters)

In [64]:
dataframe_report

,model_name,model_type,method_preprocessing,embedding_model,chunk_size,chunk_overlap,context_length,chunk_technique,temperature,gpu_layers,search_kwargs,size_model,size_answer,reorder,quantization,device,rate,rate_table,rate_text,time


In [100]:
def calculationRate(csv_filename):
    df = pd.read_csv(csv_filename,sep=';',encoding='utf-8')
    # creating list of sentences
    sentences_list_result= df['ResultLLM']
    print('sentences_list_result : ', sentences_list_result)
    sentences_list_baseline= df['Baseline']
    #takin embedding of list
    embeddings_result=  model.encode(sentences_list_result)
    embeddings_baseline = model.encode(sentences_list_baseline)

    rate = 0
    rateText = 0
    rateTable = 0
    similarity_rate=0.7
    list_questions_defaillantes = []
    for row in range(1,len(embeddings_result)):
        result = cosine_similarity(embeddings_result[row-1:row],embeddings_baseline[row-1:row]) 
        if result>similarity_rate:
            rate=rate+1
            if row>=(len(embeddings_result)-6):
                rateTable = rateTable+1
            else:
                rateText = rateText+1
        else:
            list_questions_defaillantes.append([sentences_list_result[row],sentences_list_baseline[row]])
        '''print('len(embeddings_result)-6 ',len(embeddings_result)-6)
        print('len(embeddings_result) ',len(embeddings_result))
        print('rateTable : ',rateTable)
        print('rateText : ',rateText)
        print('rate : ',rate)'''
    return rate/len(embeddings_result), rateText/(len(embeddings_result)-6), rateTable/6

In [49]:
import csv

In [98]:
def fill_out_static_paramaters(csv_file_parameters,dataframe_report):
    df = pd.read_csv(csv_file_parameters, delimiter=',')
    #print('size df.columns : ', len(df.columns))
    #print('size dataframe_report.columns : ', len(dataframe_report.columns))
    #print(dataframe_report)
    
    for col in df.columns:
        #print('col = ', col)
        #print(df[col].values)
        dataframe_report[col]=df[col].values
    return dataframe_report

In [99]:
def fill_out_rates(csv_file,dataframe_report):
    rate,rateText,rateTable=calculationRate(csv_file)
    dataframe_report['rate']=rate
    dataframe_report['rate_table']=rateTable
    dataframe_report['rate_text']=rateText
    return dataframe_report

In [103]:
csv_file_parameters = './results/parameterllama-2-13b-chat.Q4_K_M.gguf75690919-de1b-43ba-a4e1-ccd3cd45592c.csv'
csv_file = './results/llama-2-13b-chat.Q4_K_M.gguf.csv'

In [100]:
fill_out_static_paramaters(csv_file_parameters,dataframe_report)

In [101]:
dataframe_report

,model_name,model_type,method_preprocessing,embedding_model,chunk_size,chunk_overlap,context_length,size_model,size_answer,reorder,quantization,device,rate,rate_table,rate_text,chunk_technique,temperature,gpu_layers,search_kwargs
0,llama-2-13b-chat.Q4_K_M.gguf,llama,PaddleOCR,BAAI/bge-large-en-v1.5,200,64,2048,7B,200,True,GGUF,MPS,0,0,0,RecursiveTextSplitter,0.1,20,5


In [104]:
dataframe_report = fill_out_rates(csv_file,dataframe_report)

In [55]:
dataframe_report

,model_name,model_type,method_preprocessing,embedding_model,chunk_size,chunk_overlap,context_length,size_model,size_answer,reorder,quantization,device,rate,rate_table,rate_text,time


In [26]:
import os

In [103]:
path_results = './results'
path_parameters = './parameters'
def create_df_results(path_results,path_parameters):
    dataframe_report = pd.DataFrame(columns=parameters)
    row=0
    for filename in os.listdir(path_results):
        print(filename)
        dataframe_report.loc[row, :] = parameters
        dataframe_report.iloc[row] = fill_out_static_paramaters(path_parameters+'/parameter'+filename,dataframe_report.iloc[row])
        dataframe_report.iloc[row] = fill_out_rates(path_results+'/'+filename,dataframe_report.iloc[row])
        row = row +1
    return dataframe_report

In [104]:
dataframe_report = create_df_results(path_results,path_parameters)

openchat_3.5.Q4_K_M.gguf8a3d19e3-332e-4a06-80bb-52df59a801de.csv
sentences_list_result :  0      BART is a model that uses concatenated questi...
1      Some advantages of BART's setup include its s...
2      BART handles fine-tuning for machine translat...
3      The text does not provide specific informatio...
4      BART performs comparably to other pretraining...
5      BART's architecture differs from BERT and GPT...
6      The key advantage of BART's setup is that it ...
7      The purpose of the in-filling scheme in BART ...
8      BART performs well in fine-tuned text generat...
9      BART brings a 1.1 BLEU increase over a back-t...
10     The text does not provide specific pre-traini...
11     BART uses token masking in its pre-training o...
12     The text does not provide specific informatio...
13     The significance of token masking in BART's p...
14     BART performs similarly to other models on SQ...
15     The key trends observed in the results of pre...
16     The exp

In [105]:
dataframe_report

,model_name,model_type,method_preprocessing,embedding_model,chunk_size,chunk_overlap,context_length,chunk_technique,temperature,gpu_layers,search_kwargs,size_model,size_answer,reorder,quantization,device,rate,rate_table,rate_text,time
0,openchat_3.5.Q4_K_M.gguf,mistral,PaddleOCR,BAAI/bge-large-en-v1.5,200,64,2048,RecursiveTextSplitter,0.1,20,5,7B,200,True,GGUF,MPS,0.76,0.166667,0.947368,306.62558579444885
1,openchat_3.5.Q4_K_M.gguf,mistral,PaddleOCR,BAAI/bge-large-en-v1.5,200,64,2048,RecursiveTextSplitter,0.1,20,5,7B,200,True,GGUF,MPS,0.64,0.0,0.842105,298.12156772613525
2,llama-2-13b-chat.Q4_K_M.gguf,llama,PaddleOCR,BAAI/bge-large-en-v1.5,200,64,2048,RecursiveTextSplitter,0.1,20,5,7B,200,True,GGUF,MPS,0.72,0.166667,0.894737,573.254804611206
3,starling-lm-7b-alpha.Q4_K_M.gguf,mistral,PaddleOCR,BAAI/bge-large-en-v1.5,200,64,2048,RecursiveTextSplitter,0.1,20,5,7B,200,True,GGUF,MPS,0.76,0.166667,0.947368,417.3041818141937
